# Introduction to the DP0.3
<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
Contact author(s): Bob Abel, Douglas Tucker, and Melissa Graham<br>
Last verified to run: 2024-12-17 <br>
LSST Science Piplines version: Weekly 2024_50 <br>
Container size: medium <br>
Targeted learning level: beginner <br>

**Description:** An overview of the contents of the DP0.3 moving object catalogs.

**Skills:** Use the TAP service and ADQL to access the DP0.3 tables.

**LSST Data Products:** TAP dp03_catalogs.

**Packages:** `lsst.rsp.get_tap_service`

**Credit:**  The DP0.3 data set was generated by members of the Rubin Solar System Pipelines and Commissioning teams, with help from the LSST Solar System Science Collaboration, in particular: Pedro Bernardinelli, Jake Kurlander, Joachim Moeyens, Samuel Cornwall, Ari Heinze, Steph Merritt, Lynne Jones, Siegfried Eggl, Meg Schwamb, and Mario Jurić.

Mario Jurić provided essential assistance in the initial stages of creating this notebook.  The notebook authors would also like to acknowledge help from Leanne Guy, Pedro Bernardinelli, Sarah Greenstreet, Megan Schwamb, Brian Rogers, Niall McElroy, and Jake Vanderplass, among others.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook demonstrates how to access the simulated Data Preview 0.3 (DP0.3) data set in the Rubin Science Platform.

For the DP0.3 simulation, only moving objects were simulated, and only catalogs were created (there are no images).
The DP0.3 simulation is *entirely independent of and separate from* the DP0.2 simulation.

DP0.3 is a hybrid catalog that contains both real and simulated Solar System objects (asteroids, near-earth objects, Trojans, trans-Neptunian objects, comets, and even a simulated spaceship... but no major planets nor the Moon).
See the <a href="https://dp0-3.lsst.io">DP0.3 documentation</a> for more information about how the hybrid catalog was created.

In Rubin Operations, these tables would be constantly changing, updated every day with the results of
the previous night's observations.
However, for DP0.3, both a static 1-year catalog and a static 10-year catalog have been simulated.

> **Notice:** To re-iterate, for DP0.3, there are tables available for both the 1-year and 10-year catalogs.  For the remainder of this notebook, though, unless otherwise noted, we will just consider the tables for the 10-year catalog.  Users are encouraged to explore the tables for the 1-year catalog on their own.


### 1.1. Package Imports

Import general python packages and the Rubin TAP service utilities.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
import pandas as pd
from lsst.rsp import get_tap_service

## 2. Create the Rubin SSO TAP Service client

The DP0.3 data sets are available via the Table Access Protocol (TAP) service,
and can be accessed with ADQL (Astronomical Data Query Language) statements.

TAP provides standardized access to catalog data for discovery, search, and retrieval.
Full <a href="http://www.ivoa.net/documents/TAP">documentation for TAP</a> is provided by the International Virtual Observatory Alliance (IVOA).
ADQL is similar to SQL (Structured Query Langage).
The <a href="http://www.ivoa.net/documents/latest/ADQL.html">documentation for ADQL</a> includes more information about syntax and keywords.

Get an instance of the SSO TAP service, and assert that it exists.

> **Notice:** The DP0.3 TAP service is called `ssotap` (whereas `tap` is used for DP0.2).

In [ ]:
service = get_tap_service("ssotap")
assert service is not None

### 2.1. What `ssotap` schemas are available?

Create an ADQL query to select all (\*) available schemas, and use the TAP service to execute the query.

Use `to_table` to convert to an `astropy` table, which will display the results in a user-friendly way.

In [ ]:
query = "SELECT * FROM tap_schema.schemas"
results = service.search(query).to_table()
results

### 2.2 What DP0.3 tables are available?

There are four tables for the 1-year and 10-year simulation: 
`MPCORB`, `SSObject`, `SSSource`, and `DiaSource`.

Find descriptions of the tables and their schema, plus
information and advice about accessing and querying the DP0.3 tables
(including which columns are <a href="https://dp0-3.lsst.io/data-products-dp0-3/table-access-and-queries.html#unpopulated-columns">currently unpopulated</a>),
in the <a href="https://dp0-3.lsst.io/data-products-dp0-3/index.html#dp0-3-data-products-definition-document-dpdd">DP0.3 data products definitions documentation</a>.

This tutorial will explore the tables individually, with table joins to be
demonstrated in future tutorials (see also <a href="https://dp0-3.lsst.io/data-products-dp0-3/table-access-and-queries.html#table-joins">this advice on DP0.3 table joins</a>).

Select all available tables and display information about them.

In [ ]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name LIKE 'dp03_catalogs%' " \
        "ORDER BY table_index ASC"
results = service.search(query).to_table()
results

## 3. The `MPCORB` table

During Rubin Operations, Solar System Processing will occur in the daytime, after a night of observing.
It will link together the difference-image detections of moving objects and report discoveries
to the <a href="https://minorplanetcenter.net">Minor Planet Center</a> (MPC),
as well as compute derived properties (magnitudes, phase-curve fits, coordinates in various systems).

The MPC will calculate the orbital parameters and these results will be passed back to Rubin, and stored
and made available to users as the `MPCORB` table 
(the other derived properties are stored in the other three tables explored below).
The DP0.3 `MPCORB` table is a simulation of what this data product will be like after 10 years of LSST.

> **Notice:** The MPC contains all reported moving objects in the Solar System, and is not limited to those detected by LSST. Thus, the `MPCORB` table will have more rows than the `SSObject` table.

> **Notice:** For DP0.3, there was no fitting done by the MPC and the MPCORB table is the orbital elements used in the simulation (the `MPCORB` table is a truth table).

For more information about Rubin's plans for Solar System Processing, see Section 3.2.2 of the 
<a href="https://docushare.lsstcorp.org/docushare/dsweb/Get/LSE-163/LSE-163_DataProductsDefinitionDocumentDPDD.pdf">Data Products Definitions Document</a>.
Note that there remain differences between Table 4 of the DPDD, which contain the anticipated schema 
for the moving object tables, and the DP0.3 table schemas.

### 3.1. Size

Use the TAP service to count of the number of rows in the `MPCORB` table.

In [ ]:
results = service.search("SELECT COUNT(*) FROM dp03_catalogs_10yr.MPCORB")
results.to_table()

There are 14.5 million rows in the `MPCORB` table.

### 3.2. Columns

Use the TAP service to query for the column information from `MPCORB`.

Print the results as a `pandas` table.

In [ ]:
results = service.search("SELECT column_name, datatype, description, "
                         "       unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp03_catalogs_10yr.MPCORB'")
results.to_table().to_pandas()

In some cases, the column descriptions cut off in the table above.

Execute the following to see, for example, the full description for the `mpcDesignation` column.

In [ ]:
results['description'][10]

### 3.3. Retrieve a random subset

To retrieve a random subset of rows, make use of the fact that `ssObjectId` is a randomly assigned 64-bit long unsigned integer. 
Since ADQL interprets a 64-bit long unsigned integer as a 63-bit _signed_ integer, 
these range from a very large negative integer value to a very large positive integer value.
This will be fixed in the future so that all identifiers are positive numbers.

> **Notice:** By using `ssObjectId`, the following query returns a random subset of `MPCORB` rows *that are associated with a row in the `SSObject` table*. In other words, this limits the query to only retrieve moving objects from the `MPCORB` table that have been detected by LSST.

First, figure out the full range of `ssObjectId` values by using the ADQL `MIN` and `MAX` functions.

In [ ]:
results = service.search("SELECT min(ssObjectId), max(ssObjectId)"
                         "FROM dp03_catalogs_10yr.MPCORB")
results.to_table()

Define a search range for `ssObjectId` that would return no more than 1% of all objects in `MPCORB`.  We will do this by estimating a new minimum `ssObjectId` that is 1% _below_ the maximum `ssObjectId` for the full range of `ssObjectId` values.

> **Notice:** Since the _range_ of `ssObjectId`'s (-9223370383071521539 --> +9223370875126069107) is much larger than the number of _rows_ in the `MPCORB` table (14600302), we don't expect to get _exactly_ 1% of the rows from `MPCORB` via this method, but we should get approximately 1%, as long as the `ssObjectId` values are distributed reasonably uniformly over their large range.


In [ ]:
min_val = int(results[0].get('min1'))
max_val = int(results[0].get('max2'))
print('Full range: ', min_val, max_val)

min_val = int(max_val - 0.01*(max_val-min_val))
print('Search range: ', min_val, max_val)

Execute the search, and retrieve all (\*) columns from the `MPCORB` table.

Define query.

In [ ]:
query = """SELECT * FROM dp03_catalogs_10yr.MPCORB 
              WHERE ssObjectId BETWEEN 9038903462544093184 
              AND 9223370875126069107"""

Submit asynchronous query.

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
results = job.fetch_result()
assert job.phase == 'COMPLETED'

Store the results in `df` as a `pandas` dataframe.

In [ ]:
df = results.to_table().to_pandas()

Display the resulting dataframe. Note that it is automatically truncated for both columns and rows 
(look for the "..." halfway down, and halfway across).

In [ ]:
df

As we see, 144,472 rows were returned, which -- _as expected_ -- is almost (but not exactly) 1% of the 14,462,388 rows in the `MPCORB` table.  

> **Notice:** There are several columns that currently contain `NaN` (not a number) values.
For the simulated DP0.3 data these columns might be replaced in the near future, 
and for real data releases there will not be all-`NaN` columns.  If desired, users can drop all-`NaN` columns with, e.g., `df.dropna(axis=1, how='all', inplace=True)`.
However the better practice is to understand the columns and retrieve only what you are going to use.

**Optional:** use the `pandas` dataframe `info` function to learn more about the values in the retrieved columns.
Uncomment the following code cell (remove the # symbol) and execute the cell to see the info on `df`.

In [ ]:
# df.info()

**Optional:** use the `pandas` fuction `describe` to display statistics for the 
numerical columns in the dataframe (count, mean, standard deviation, etc.).

In [ ]:
# df.describe()

### 3.4. Plot histograms of selected columns

Wikipedia provides a decent <a href="https://en.wikipedia.org/wiki/Orbital_elements">beginner-level guide to orbital elements</a>.

For a quick reference, distributions are shown below for five key orbital elements
and the absolute $H$ magnitude (see Section 4.4 for a description of $H$).

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10, 6), sharey=False)
ax[0, 0].hist(df['e'], bins=100, log=True)
ax[0, 0].set_xlabel('Eccentricity')
ax[0, 0].set_ylabel('log(Number)')
ax[0, 1].hist(df['incl'], bins=100, log=True)
ax[0, 1].set_xlabel('Inclination (deg)')
ax[0, 1].set_ylabel('log(Number)')
ax[0, 2].hist(df['mpcH'], bins=100, log=True)
ax[0, 2].set_xlabel('Absolute Magnitude, H (mag)')
ax[0, 2].set_ylabel('log(Number)')
ax[1, 0].hist(df['node'], bins=50)
ax[1, 0].set_xlabel('Longitude of Ascending Node (deg)')
ax[1, 0].set_ylabel('Number')
ax[1, 0].set_ylim(0,3500)
ax[1, 1].hist(df['peri'], bins=50)
ax[1, 1].set_xlabel('Argument of Perihelion (deg)')
ax[1, 1].set_ylabel('Number')
ax[1, 1].set_ylim(0,3500)
ax[1, 2].hist(df['q'], bins=100, log=True)
ax[1, 2].set_xlabel('Perihelion Distance (au)')
ax[1, 2].set_ylabel('log(Number)')
fig.suptitle('Histograms for Key Orbital Elements')
fig.tight_layout()
plt.show()

> **Figure 1**: Six histograms showing the log number of solar system objects in the `MPCORB` table as a function of their eccentricity, inclination, absolute H magnitude, longitude of the ascending node, argument of perihelion, and perihelion distance (from upper left to lower ight).

## 4. The `SSObject` table

During Rubin Operations, Prompt Processing will occur during the night, detecting sources in 
difference images (`DiaSources`, see Section 6) and associating them into static-sky transients
and variables (`DiaObjects`, not included in DP0.3).

The Solar System Processing which occurs in the daytime, after a night of observing,
links together the `DiaSources` for moving objects into `SSObjects`.
Whereas the `MPCORB` table contains the orbital elements for these moving objects,
the `SSObjects` contains the Rubin-measured properties such as phase curve fits and absolute magnitudes.

> **Notice:** no artifacts or spurious difference-image sources have been injected into the DP0.3 catalogs.

> **Notice:** although there are columns for them, no _u-_ or _Y_-band data were simulated for DP0.3.  Unless noted, we will ignore _u-_ or _Y_-band data for the remainder of this notebook.

### 4.1. Size

Use the ADQL count function to return the size.

In [ ]:
results = service.search("SELECT COUNT(*) from dp03_catalogs_10yr.SSObject")
results.to_table().to_pandas()

The DP0.3 data set contains 4.4 million solar system objects detected by Rubin.

This is less than the 14.5 million objects in the `MPCORB` catalog.
It is left as an exercise for the learner in Section 7 to determine the characteristics of those 
objects from the `MPCORB` table are missing from the `SSObject` table.

### 4.2. Columns

**Option:** display a list of column names, data types, descriptions, and units.

In [ ]:
# results = service.search("SELECT column_name, datatype, description, "
#                          "unit from TAP_SCHEMA.columns "
#                          "WHERE table_name = 'dp03_catalogs_10yr.SSObject'")
# results.to_table().to_pandas()

### 4.3. Retrieve a random subset

Use essentially the same query as was used for the `MPCORB` table, above.

Define query.

In [ ]:
query = """SELECT * FROM dp03_catalogs_10yr.ssObject 
                WHERE ssObjectId BETWEEN 9038903462544093184 
                AND 9223370875126069107"""


Submit asynchronous query.

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
results = job.fetch_result()
assert job.phase == 'COMPLETED'

Store the results in df as a pandas dataframe.

In [ ]:
df = results.to_table().to_pandas()

Display the resulting dataframe.

In [ ]:
df

There are 44611 rows.

**Options:** use the `info` or `describe` functions on the `pandas` dataframe to learn more about the retrieved results.

In [ ]:
# df.info()

In [ ]:
# df.describe()

### 4.4. Plot a color-color diagram

In the displayed dataframe above, it appears that for many `SSObjects` the phase-curve fits to derive 
absolute magnitudes were not successful, as the `<band>_H` (where `<band>` is a band u, g, r, i, z, or y) are `NaN`.  (Recall in  particular that no _u-_ or _y_-band data were simulated for DP0.3.)

Before calculating and plotting the colors, drop all of the rows for which the phase-curve fits were not successful for g, r, i, and/or z bands.

In [ ]:
df.dropna(subset=['g_H', 'r_H', 'i_H', 'z_H'], inplace=True)
df.reset_index(inplace=True)
print('Number of rows after dropping rows: %d' % len(df))

For Solar System objects, absolute magnitudes are defined to be for an object 1 au from the Sun and 1 au 
from the observer, and at a phase angle (the angle Sun-object-Earth) of 0 degrees.

Absolute magnitudes are derived by correcting for distance, fitting a function to the relationship between 
absolute magnitude and phase, and evaluating the function at a phase of 0 deg.

The process for fitting phase curves will be covered in another tutorial.

Calculate colors in the Rubin filters for the `SSObjects` that have absolute magnitudes.

In [ ]:
df['gr'] = df['g_H'] - df['r_H']
df['ri'] = df['r_H'] - df['i_H']
df['iz'] = df['i_H'] - df['z_H']

Plot color-color diagrams as 2-dimensional histograms (heatmaps).

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3))

h = ax[0].hist2d(df['gr'], df['ri'],
                 bins=(np.arange(-0.5, 1.5, 0.01),
                       np.arange(-0.5, 0.75, 0.01)),
                 norm='log')
ax[0].set_xlabel('g - r')
ax[0].set_ylabel('r - i')
ax[0].grid()
plt.colorbar(h[3])

h = ax[1].hist2d(df['ri'], df['iz'],
                 bins=(np.arange(-0.5, 0.75, 0.01),
                       np.arange(-0.75, 0.5, 0.01)),
                 norm='log')
ax[1].set_xlabel('r - i')
ax[1].set_ylabel('i - z')
ax[1].grid()
plt.colorbar(h[3])

fig.suptitle('Color-Color Plots for the SSObject Catalog')
fig.tight_layout()
plt.show()

> **Figure 2:** Two panels showing color-color plots for the retrieved subset of the `SSObject` catalog. At left, the $r-i$ vs. $g-r$ color and at right, the $i-z$ versus $r-i$ color. In both panels, the color distribution is bimodal.

As shown in the figure above, there are two colors used in the simulation - but this is not the case for real Solar System objects.
These plots will look very different in the future, when they are made with real Rubin data.

## 5. The `SSSource` table

As described above, Solar System Processing links together the `DiaSources` (detections in the
individual difference images) from moving objects into `SSObjects`. 

The `SSSource` table contains the 2-d (sky) coordinates and 3-d distances and velocities 
for every `SSObject` at the time of every LSST observation of that `SSObject`.
The `SSSource` and `DiaSource` tables are 1:1, as they each contain data *per observation*,
whereas `SSObject` contains data *per object*.

### 5.1. Size

It can take up to a minute to retrieve the size of the `SSSource` catalog.

Define query.

In [ ]:
query = "SELECT COUNT(*) from dp03_catalogs_10yr.SSSource"


Submit asynchonous query.

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
results = job.fetch_result()
assert job.phase == 'COMPLETED'

Display results as a pandas data frame. 

In [ ]:
results.to_table().to_pandas()

**_This table contains over 653 million sources!_**

### 5.2. Columns

**Option:** print the column information for the `SSSource` table.

In [ ]:
# results = service.search("SELECT column_name, datatype, description, "
#                          "unit from TAP_SCHEMA.columns "
#                          "WHERE table_name = 'dp03_catalogs_10yr.SSSource'")
# results.to_table().to_pandas()

### 5.3. Retrieve data for one `SSObject`

It is possible to obtain the `SSSource` data for a set of `SSObjects`.
For example, to retrieve all `SSSources` for the `SSObjects` retrieved in Section 4,
use a query such as: 

```
SELECT * FROM dp03_catalogs_10yr.SSSource
WHERE ssObjectId BETWEEN 9038903462544093184 AND 9223370875126069107
```

However, the better way to demonstrate the data in the `SSSource` table is to look at just one `SSObject`,
and the one with an `ssObjectId` = `6793512588511170680` is a fun choice.

Retrieve the heliocentric (sun-centered) and topocentric (observatory-centered) X and Y coordinates.

Define the query and run the asynchronous query.

In [ ]:
query = """SELECT heliocentricX, heliocentricY, 
              topocentricX, topocentricY, ssObjectId 
              FROM dp03_catalogs_10yr.SSSource 
              WHERE ssObjectId = 6793512588511170680"""

job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
results = job.fetch_result()
assert job.phase == 'COMPLETED'

df_xy = results.to_table().to_pandas()
print('Retrieved ', len(df_xy), ' rows.')

**Options** to display the table in full or use the `info` or `describe` functions.

In [ ]:
# df_xy

In [ ]:
# df_xy.info()

In [ ]:
# df_xy.describe()

### 5.3. Plot the locations of one `SSObject`

Plot the locations of the selected `SSObject` at the time of every 
LSST observation using the X and Y heliocentric (Sun-centered; orange star) 
and topocentric (observatory-centered; blue circle) coordinates.
This can be considered a projection of the orbit into the plane of the Solar System.

Notice how the points are not regularly spaced.
This is because there is one point per LSST observation of the object,
and in some years it receives more or fewer observations.

Notice how the points appear in an ellipse around the Sun with heliocentric coordinates (left).
This is because the selected object is in the main asteroid belt and close enough to the Sun 
to complete at least on orbit during the 10-year LSST survey.
Had the selected object been in the outer Solar System, or were this tutorial using
the 1-year data set, the plot below would show an arc instead of an ellipse.

The plot of the topocentric coordinates (right) does not appear elliptical because
the motion of the Earth with respect to the object over the 10 years of the LSST
is imprinted into the data. 
For the topocentric coordinates, the Earth's rotation also contributes, 
but it is a much smaller effect on the scale of these plots.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].grid()
ax[0].plot(df_xy['heliocentricX'], df_xy['heliocentricY'],
           'o', ms=4, mew=0, color='black')
ax[0].plot(0, 0, '*', ms=15, color='darkorange')
ax[0].set_xlabel('heliocentric X (au)')
ax[0].set_ylabel('heliocentric Y (au)')
ax[0].set_title('Heliocentric Coordinates')
ax[1].grid()
ax[1].plot(df_xy['topocentricX'], df_xy['topocentricY'],
           'o', ms=4, mew=0, color='black')
ax[1].plot(0, 0, 'o', ms=15, color='dodgerblue')
ax[1].set_xlabel('topocentric X (au)')
ax[1].set_ylabel('topocentric Y (au)')
ax[1].set_title('Topocentric Coordinates')
fig.suptitle('XY Path of 6793512588511170680')
fig.tight_layout()
plt.show()

> **Figure 3:** The two-dimensional projection of the location of the selected solar system object into the XY plan in heliocentric (sun-centered; left) and topocentric (earth-centered; right) coordinates. At left, the Sun is represented by an orange star, and at right, the Earth is represented by a blue circle. In both plots the object is represented by small black circles.

## 6. The `DiaSource` catalog

Last but definitely not least, the `DiaSource` table - which is actually the first to be generated
by the Prompt Processing pipeline.
This table will contain measurements for all sources detected with a signal-to-noise ratio of at least 5 
in a difference image, including moving and static-sky sources.

However, for simulated DP0.3 the `DiaSource` table contains *only moving objects*, and no static sky time-domain objects (and no detector artifacts).

For DP0.3 the simulated `DiaSource` table contains only a subset of the columns that the
real `DiaSource` table will have; see Table 1 of the Rubin 
<a href="https://docushare.lsstcorp.org/docushare/dsweb/Get/LSE-163/LSE-163_DataProductsDefinitionDocumentDPDD.pdf">Data Products Definitions Document</a>.
Furthermore, the `DiaSource` table contains a few extra truth columns, such as `raTrue`, `decTrue`, `magTrue`.

For DP0.3, no photometric variability due to, e.g., the rotation of non-spherical bodies, collisions,
or outgassing events were simulated. 
All evolution in apparent magnitudes with time are due to the phase curve, which is explored
in another tutorial.

### 6.1. Size

**Option** to retrieve the size of the `DiaSource` table (it is 1:1 with the `SSSource` table, so the same size).

In [ ]:
# results = service.search("SELECT COUNT(*) from dp03_catalogs_10yr.DiaSource")
# results.to_table().to_pandas()


### 6.2. Columns

**Option** to print the column information.

In [ ]:
# results = service.search("SELECT column_name, datatype, description, "
#                          "unit from TAP_SCHEMA.columns "
#                          "WHERE table_name = 'dp03_catalogs_10yr.DiaSource'")
# results.to_table().to_pandas()

### 6.3. Retrieve data for one `SSObject`

Similar to what we did for `SSSource` data in Section 5, it is possible to obtain the `DiaSource` data for a set of `SSObjects`.  For example, to retrieve all `DiaSources` for the `SSObjects` retrieved in Section 4,
use a query such as: 

```
SELECT * FROM dp03_catalogs_10yr.DiaSource
WHERE ssObjectId BETWEEN 9038903462544093184 AND 9223370875126069107
```

However, as we saw for the `SSSource` data in Section 5, the better way to demonstrate the data in the `DiaSource` table is to look at just one `SSObject`.
Here, as in Section 5, we will use the one with an `ssObjectId` = `6793512588511170680`.

Retrieve the right ascension, declination, and the <a href="https://en.wikipedia.org/wiki/International_Atomic_Time">TAI</a> at the exposure's midpoint (MJD) of the observation.

In [ ]:
query = """SELECT ra, dec, midpointMjdTai 
              FROM dp03_catalogs_10yr.DiaSource 
              WHERE ssObjectId = 6793512588511170680"""

job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
results = job.fetch_result()
assert job.phase == 'COMPLETED'

df = results.to_table().to_pandas()
df

### 6.4. Plot time-domain data for one `SSObject`

Here, we plot the equatorial coordinates for all observations, colored by the MJD of the observation.

This plot shows the location of the object on the sky, as seen from Earth.
Recall from the plots in Section 5 that this object is only about 2 au from the Sun,
and so is at the inner edge of the Main Asteroid Belt.
Parallax from the Earth's orbit contributes to the appearance of this plot.

In [ ]:
cmap = colormaps['viridis']
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
im = ax.scatter(df['ra'], df['dec'], c=df['midpointMjdTai'], cmap=cmap)
ax.set_xlabel('ra (deg)')
ax.set_ylabel('dec (deg)')
fig.suptitle("Path of one SSObject in equatorial coordinates over 10 years")
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='TAI')
ax.grid(True)

> **Figure 4:** The declination (dec) versus right ascencion (ra) in decimal degrees of the difference-image detections of the selected solar system object. The points are colored by the date (TAI) of the observation to illustrate the object's path in the sky over time.

Use of the magnitude and filter data retrieved from the `DiaSource` table is left as
an exercise for the learner in Section 7.

## 7. Exercises for the learner


**1. Calculate the semi-major axis for a subset of objects.**

The orbital element of semi-major axis ($a$) is not pre-computed in the `MPCORB` table
because it can be derived from the orbit’s ellipticiy ($e$) and perihelion distance ($q$)
with $a = q/(1-e)$.

Copy the query from Section 3.3 and alter it to retrieve only $e$ and $q$ for a subset of objects.
Then add a column to the results table for semi-major axis and plot eccentricity 
versus semi-major axis.

Hints:
 1. Restrict the query to objects with eccentricities between 0 and 1.
 2. `df['a'] = df['q'] / (1.0 - df['e'])`

**2. How does an SSObject's magnitude change with time?**

In Section 6.4, the magnitude and filter for all detections of the `SSObject` in the
`DiaSource` table were not retrieved; alter the query to retrieve `mag` and `band`,
and plot the magnitude as a function of time for the filter of your choice
(recall: there will be no *u*- or *y*-band magnitudes).
Note that the only simulated cause of photometric changes in DP0.3 objects is the distance from Earth and the phase curve.
Correcting for distance, and the applications phase curves, are covered in another tutorial.